### TensorFlow 2.0

In [2]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import datasets
from tensorflow.keras.preprocessing.image import ImageDataGenerator

### Hyperparameter Tunning

In [27]:
num_epochs = 5
batch_size = 32
learning_rate = 0.001
dropout_rate = 0.5
input_shape = (32,32,3)
num_classes = 10

### Preprocess

In [6]:
import pandas as pd
import numpy as np

In [7]:
train_df = pd.read_csv('train_dataset.csv')
test_df = pd.read_csv('test_dataset.csv')

train_df.head(2)

,name,class
0,../dataset/cifar/train\0_frog.png,frog
1,../dataset/cifar/train\10000_automobile.png,automobile


In [9]:
train_datagen = ImageDataGenerator(
    rescale=1./255.,
    width_shift_range=0.3,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(
    rescale=1./255.
)

In [10]:
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col='name',
    y_col='class',
    target_size=input_shape[:2],
    batch_size=batch_size
)

test_generator = test_datagen.flow_from_dataframe(
    test_df,
    x_col='name',
    y_col='class',
    target_size=input_shape[:2],
    batch_size=batch_size
)

Found 50000 validated image filenames belonging to 10 classes.
Found 10000 validated image filenames belonging to 10 classes.


### Build Model

In [19]:
inputs = layers.Input(input_shape)
net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [20]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

### Training

In [28]:
model.fit_generator(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=num_epochs,
    validation_data=test_generator,
    validation_steps=len(test_generator)
)

C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
1563/1563 [==============================] - 265s 169ms/step - loss: 1.9116 - accuracy: 0.2875 - val_loss: 1.4086 - val_accuracy: 0.4828
Epoch 2/5
1563/1563 [==============================] - 224s 144ms/step - loss: 1.5246 - accuracy: 0.4467 - val_loss: 1.2997 - val_accuracy: 0.5287
Epoch 3/5
1563/1563 [==============================] - 189s 121ms/step - loss: 1.3883 - accuracy: 0.4961 - val_loss: 1.2008 - val_accuracy: 0.5608
Epoch 4/5
1563/1563 [==============================] - 184s 118ms/step - loss: 1.3112 - accuracy: 0.5231 - val_loss: 1.1477 - val_accuracy: 0.5849
Epoch 5/5
1563/1563 [==============================] - 184s 117ms/step - loss: 1.2682 - accuracy: 0.5447 - val_loss: 1.0222 - val_accuracy: 0.6287
